<a href="https://colab.research.google.com/github/akalakheti/estimator/blob/master/Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu==2.0.0

(Importing the required libraries)

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import layers

We will be using fashion_mnist dataset to train and test our model created on tf.keras and tf.estimator.

In [0]:
(train_data, train_label), (test_data,test_label) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


A function to normalize the image for train and test test and converting the data type of label to 'int32' as the estimator model requires so.

In [0]:
def norm(image, label):
  img= image/255.0
  lbl = np.array(label, dtype='int32')
  return img, lbl

train_data, train_l = norm(train_data, train_label)
test_data,test_l = norm(test_data,test_label)

# Creating the model using tf.estimator.

Creating a training input function. It provides a quick way to batch, shuffle and repeat the data.

In [0]:
train_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
            x={"x":train_data}, # Data stored in a dictionary.
            y= train_l, #label
            num_epochs=None,
            batch_size=50,
            shuffle=True)

Similarly, creating test input function.

In [0]:
test_fn= tf.compat.v1.estimator.inputs.numpy_input_fn(
            x={"x":test_data}, # Data stored in a dictionary.
            y= test_l, #label
            num_epochs=1,
            shuffle=False)

To pass our data to the model created using estimator, we have to create something called feature columns. 

In [0]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[28,28])]

Now, we create the actual model from estimatory.

In [0]:
model_estimator = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                             hidden_units=[256,32], #number of nodes in hidden layers
                                             optimizer=tf.keras.optimizers.Adam(1e-4),
                                             n_classes=10,dropout=0.15,
                                             model_dir="/estimator_model", #path to save the model
                                             loss_reduction=tf.losses.Reduction.SUM)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/estimator_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe9aab98080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Training of model_estimator.

In [0]:
model_estimator.train(input_fn=train_fn, steps=10000)

Evaluating the trained model_estimator.

In [0]:
test_acc = model_estimator.evaluate(input_fn=test_fn)['accuracy']

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-07T08:44:47Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /estimator_model/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-12-07-08:44:47
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.8668, average_loss = 0.36681268, global_step = 10000, lo

In [0]:
test_acc

0.8668

# Creating the model using tf.keras

Now, Let's create a model with same architecture using tf.keras. 

In [0]:
model_keras = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(256, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model_keras.compile(optimizer = 'adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])


Training the tf.keras model.

In [0]:
model_keras.fit(train_data, train_l,epochs=9)

Train on 60000 samples
Epoch 1/9
60000/60000 [==============================] - 5s 80us/sample - loss: 0.4915 - accuracy: 0.8262
Epoch 2/9
60000/60000 [==============================] - 4s 74us/sample - loss: 0.3666 - accuracy: 0.8667
Epoch 3/9
60000/60000 [==============================] - 4s 75us/sample - loss: 0.3299 - accuracy: 0.8793
Epoch 4/9
60000/60000 [==============================] - 4s 73us/sample - loss: 0.3084 - accuracy: 0.8864
Epoch 5/9
60000/60000 [==============================] - 4s 73us/sample - loss: 0.2881 - accuracy: 0.8922
Epoch 6/9
60000/60000 [==============================] - 4s 74us/sample - loss: 0.2737 - accuracy: 0.8980
Epoch 7/9
60000/60000 [==============================] - 5s 78us/sample - loss: 0.2618 - accuracy: 0.9028
Epoch 8/9
60000/60000 [==============================] - 5s 78us/sample - loss: 0.2508 - accuracy: 0.9065
Epoch 9/9
60000/60000 [==============================] - 5s 76us/sample - loss: 0.2408 - accuracy: 0.9099


Evalutaing the trained tf.keras model.

In [0]:
test_loss, test_acc = model_keras.evaluate(test_data, test_l)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [0]:
test_acc

0.8746

# Converting the tf.keras model to tf.estimator model

In [0]:
converted_estimator_model = tf.keras.estimator.model_to_estimator(
    keras_model=model_keras,
    keras_model_path=None,
    custom_objects=None,
    model_dir=None,
    config=None,
    checkpoint_format='checkpoint'
)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp195_53bh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdb7ee8dcf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
type(converted_estimator_model)

tensorflow_estimator.python.estimator.estimator.EstimatorV2

# Conclusion

The model created using tf.keras had a slight (~1%) better test accuracy than the model created using tf.estimator. I haven't compared both the model on a larger dataset, but from what I've read, tf.estimator takes an edge on larger dataset.

So, we could use tf.estimator if we want to take advantage of it's distributed training feature and if the dataset if huge. Using estimators, you could easily integrate other tensorflow tools. If we don't want above mentioned features, then, tf.keras would be a better option.